In [0]:
# Unmount the existing mount point if it exists
try:
    dbutils.fs.unmount("/mnt/bronze-ankit")
except:
    pass

# bronze mount
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope="my_scope", key="clientid"),
    "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="my_scope", key="clientsecret"),
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d1e4ebf6-743a-4c8c-93d3-8f0bf46b74b2/oauth2/token"
}

# Replace with your storage account name and container name
storage_account_name = "bpclcloud"
container_name = "bronze-ankit"

mount_point = "/mnt/" + container_name

dbutils.fs.mount(
    source=f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point=mount_point,
    extra_configs=configs
)

/mnt/bronze-ankit has been unmounted.


True

In [0]:
# bronze mount
# Unmount the existing mount point if it exists
try:
    dbutils.fs.unmount("/mnt/silver-ankit")
except:
    pass

configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope="my_scope", key="clientid"),
    "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="my_scope", key="clientsecret"),
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d1e4ebf6-743a-4c8c-93d3-8f0bf46b74b2/oauth2/token"
}

# Replace with your storage account name and container name
storage_account_name = "bpclcloud"
container_name = "silver-ankit"

mount_point = "/mnt/" + container_name

dbutils.fs.mount(
    source=f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point=mount_point,
    extra_configs=configs
)

/mnt/silver-ankit has been unmounted.


True

In [0]:
# bronze mount
try:
    dbutils.fs.unmount("/mnt/bronze-auto")
except:
    pass


configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope="my_scope", key="clientid"),
    "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="my_scope", key="clientsecret"),
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d1e4ebf6-743a-4c8c-93d3-8f0bf46b74b2/oauth2/token"
}

# Replace with your storage account name and container name
storage_account_name = "bpclcloud"
container_name = "bronze-auto"

mount_point = "/mnt/" + container_name

dbutils.fs.mount(
    source=f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point=mount_point,
    extra_configs=configs
)

/mnt/bronze-auto has been unmounted.


True

In [0]:
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

# Paths
raw_path = "/mnt/bronze-ankit/Sales_order"
bronze_path = "/mnt/bronze-auto/sales_order"  # Target folder

# Read the raw Parquet data
bronze_df = (spark.readStream
             .format("cloudFiles")
             .option("cloudFiles.format", "parquet")
             .option("cloudFiles.schemaLocation", f"{bronze_path}/_schema")  # Schema location
             .option("cloudFiles.inferColumnTypes", "true")
             .load(raw_path))

# Add the 'IngestedAt' column
bronze_df = bronze_df.withColumn("IngestedAt", current_timestamp())

# Write to the Bronze path
bronze_df.writeStream.format("delta") \
    .option("checkpointLocation", f"{bronze_path}/_checkpoint") \
    .outputMode("append") \
    .start(bronze_path)


In [0]:
# Read data from Bronze layer
bronze_data = spark.read.format("parquet").load(raw_path)



# Print schema to verify structure
display(bronze_data)


OrderID,CustomerID,ProductID,OrderDate,Quantity,UnitPrice,TotalAmount,OrderStatus,LastUpdated
1,101,201,2025-01-01T10:30:00Z,2,50.00,100.00,Pending,2025-01-01T10:30:00Z
2,102,202,2025-01-02T14:15:00Z,1,75.50,75.50,Completed,2025-01-02T14:15:00Z
3,103,203,2025-01-03T09:45:00Z,5,20.00,100.00,Shipped,2025-01-03T09:45:00Z
4,104,204,2025-01-04T16:00:00Z,3,150.75,452.25,Cancelled,2025-01-04T16:00:00Z
5,105,205,2025-01-05T11:20:00Z,10,10.50,105.00,Pending,2025-01-05T11:20:00Z
6,106,206,2025-01-06T18:30:00Z,4,99.99,399.96,Completed,2025-01-06T18:30:00Z
7,107,207,2025-01-07T13:10:00Z,7,40.00,280.00,Shipped,2025-01-07T13:10:00Z
8,108,208,2025-01-08T15:45:00Z,6,25.25,151.50,Pending,2025-01-08T15:45:00Z
9,109,209,2025-01-09T10:00:00Z,12,8.99,107.88,Completed,2025-01-09T10:00:00Z
10,10,110,2025-01-10T12:00:00Z,3,60.00,180.00,Pending,2025-01-10T12:00:00Z


In [0]:

# Read data from Silver layer
stream_data = spark.read.format("delta").load(bronze_path)

display(stream_data)

OrderID,CustomerID,ProductID,OrderDate,Quantity,UnitPrice,TotalAmount,OrderStatus,LastUpdated,_rescued_data,IngestedAt
1,101,201,2025-01-01T10:30:00Z,2,50.00,100.00,Pending,2025-01-01T10:30:00Z,null,2025-01-15T06:40:11.095Z
2,102,202,2025-01-02T14:15:00Z,1,75.50,75.50,Completed,2025-01-02T14:15:00Z,null,2025-01-15T06:40:11.095Z
3,103,203,2025-01-03T09:45:00Z,5,20.00,100.00,Shipped,2025-01-03T09:45:00Z,null,2025-01-15T06:40:11.095Z
4,104,204,2025-01-04T16:00:00Z,3,150.75,452.25,Cancelled,2025-01-04T16:00:00Z,null,2025-01-15T06:40:11.095Z
5,105,205,2025-01-05T11:20:00Z,10,10.50,105.00,Pending,2025-01-05T11:20:00Z,null,2025-01-15T06:40:11.095Z
6,106,206,2025-01-06T18:30:00Z,4,99.99,399.96,Completed,2025-01-06T18:30:00Z,null,2025-01-15T06:40:11.095Z
7,107,207,2025-01-07T13:10:00Z,7,40.00,280.00,Shipped,2025-01-07T13:10:00Z,null,2025-01-15T06:40:11.095Z
8,108,208,2025-01-08T15:45:00Z,6,25.25,151.50,Pending,2025-01-08T15:45:00Z,null,2025-01-15T06:40:11.095Z
9,109,209,2025-01-09T10:00:00Z,12,8.99,107.88,Completed,2025-01-09T10:00:00Z,null,2025-01-15T06:40:11.095Z
10,10,110,2025-01-10T12:00:00Z,3,60.00,180.00,Pending,2025-01-10T12:00:00Z,null,2025-01-15T06:40:11.095Z


In [0]:
from delta.tables import DeltaTable
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Paths
bronze_path = "/mnt/bronze-auto/sales_order"
silver_path = "/mnt/silver-ankit/sales_order"

# Read the Bronze Delta Table as a streaming DataFrame
bronze_stream_df = (spark.readStream
                    .format("delta")
                    .load(bronze_path))

# Deduplicate records in the micro-batch
def deduplicate_bronze_records(df):
    window_spec = Window.partitionBy("OrderID").orderBy("LastUpdated")
    return df.withColumn("row_number", row_number().over(window_spec)).filter("row_number = 1").drop("row_number")

# Initialize Silver Table if it doesn't exist
if not DeltaTable.isDeltaTable(spark, silver_path):
    bronze_batch_df = spark.read.format("delta").load(bronze_path)
    bronze_batch_df.write.format("delta").save(silver_path)

# Define the merge logic for CDC
def upsert_to_silver(microBatchOutputDF, batchId):
    deduplicated_df = deduplicate_bronze_records(microBatchOutputDF)
    
    silver_table = DeltaTable.forPath(spark, silver_path)
    silver_table.alias("silver").merge(
        source=deduplicated_df.alias("bronze"),
        condition="silver.OrderID = bronze.OrderID"
    ).whenMatchedUpdate(set={
        "CustomerID": "bronze.CustomerID",
        "ProductID": "bronze.ProductID",
        "OrderDate": "bronze.OrderDate",
        "Quantity": "bronze.Quantity",
        "UnitPrice": "bronze.UnitPrice",
        "TotalAmount": "bronze.TotalAmount",
        "OrderStatus": "bronze.OrderStatus",
        "LastUpdated": "bronze.LastUpdated",
        "IngestedAt": "bronze.IngestedAt"
    }).whenNotMatchedInsert(values={
        "OrderID": "bronze.OrderID",
        "CustomerID": "bronze.CustomerID",
        "ProductID": "bronze.ProductID",
        "OrderDate": "bronze.OrderDate",
        "Quantity": "bronze.Quantity",
        "UnitPrice": "bronze.UnitPrice",
        "TotalAmount": "bronze.TotalAmount",
        "OrderStatus": "bronze.OrderStatus",
        "LastUpdated": "bronze.LastUpdated",
        "IngestedAt": "bronze.IngestedAt"
    }).execute()

# Write the streaming DataFrame to the Silver Delta Table
(bronze_stream_df.writeStream
 .foreachBatch(upsert_to_silver)
 .trigger(once=True)  # Trigger the stream to run once and terminate
 .outputMode("update")
 .option("checkpointLocation", f"{silver_path}/_checkpoint")
 .start())


In [0]:
# Path to the Silver Delta Table
silver_path = "/mnt/silver-ankit/sales_order"

# Read the Silver table as a batch DataFrame
silver_df = spark.read.format("delta").load(silver_path)

# Show the data
display(silver_df)


OrderID,CustomerID,ProductID,OrderDate,Quantity,UnitPrice,TotalAmount,OrderStatus,LastUpdated,_rescued_data,IngestedAt
1,101,201,2025-01-01T10:30:00Z,2,50.00,100.00,Pending,2025-01-01T10:30:00Z,null,2025-01-15T06:40:11.095Z
2,102,202,2025-01-02T14:15:00Z,1,75.50,75.50,Completed,2025-01-02T14:15:00Z,null,2025-01-15T06:40:11.095Z
3,103,203,2025-01-03T09:45:00Z,5,20.00,100.00,Shipped,2025-01-03T09:45:00Z,null,2025-01-15T06:40:11.095Z
4,104,204,2025-01-04T16:00:00Z,3,150.75,452.25,Cancelled,2025-01-04T16:00:00Z,null,2025-01-15T06:40:11.095Z
5,105,205,2025-01-05T11:20:00Z,10,10.50,105.00,Pending,2025-01-05T11:20:00Z,null,2025-01-15T06:40:11.095Z
6,106,206,2025-01-06T18:30:00Z,4,99.99,399.96,Completed,2025-01-06T18:30:00Z,null,2025-01-15T06:40:11.095Z
7,107,207,2025-01-07T13:10:00Z,7,40.00,280.00,Shipped,2025-01-07T13:10:00Z,null,2025-01-15T06:40:11.095Z
8,108,208,2025-01-08T15:45:00Z,6,25.25,151.50,Pending,2025-01-08T15:45:00Z,null,2025-01-15T06:40:11.095Z
9,109,209,2025-01-09T10:00:00Z,12,8.99,107.88,Completed,2025-01-09T10:00:00Z,null,2025-01-15T06:40:11.095Z
10,10,110,2025-01-10T12:00:00Z,3,60.00,180.00,Pending,2025-01-10T12:00:00Z,null,2025-01-15T06:40:11.095Z


In [0]:
%sql
DESCRIBE HISTORY '/mnt/silver-ankit/sales_order';


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2025-01-15T06:50:31Z,7094078440423603,28ankit.raghu@gmail.com,MERGE,"Map(predicate -> [""(OrderID#57949 = OrderID#57696)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3581695015416350),0107-100201-c3skwga5,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 3159, numTargetBytesRemoved -> 3159, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 1744, materializeSourceTimeMs -> 354, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 735, numTargetRowsUpdated -> 1, numOutputRows -> 1, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 635)",null,Databricks-Runtime/16.1.x-scala2.12
2,2025-01-15T06:47:30Z,7094078440423603,28ankit.raghu@gmail.com,MERGE,"Map(predicate -> [""(OrderID#49449 = OrderID#48883)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3581695015416350),0107-100201-c3skwga5,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 3159, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 1285, materializeSourceTimeMs -> 306, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 536, numTargetRowsUpdated -> 0, numOutputRows -> 1, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 421)",null,Databricks-Runtime/16.1.x-scala2.12
1,2025-01-15T06:41:26Z,7094078440423603,28ankit.raghu@gmail.com,MERGE,"Map(predicate -> [""(OrderID#32907 = OrderID#32100)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3581695015416350),0107-100201-c3skwga5,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 3742, numTargetBytesRemoved -> 3742, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 14, executionTimeMs -> 2988, materializeSourceTimeMs -> 566, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1381, numTargetRowsUpdated -> 14, numOutputRows -> 14, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 14, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1011)",null,Databricks-Runtime/16.1.x-scala2.12
0,2025-01-15T06:41:20Z,7094078440423603,28ankit.raghu@gmail.com,WRITE,"Map(mode -> ErrorIfExists, statsOnLoad -> false, partitionBy -> [])",null,List(3581695015416350),0107-100201-c3skwga5,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 14, numOutputBytes -> 3742)",null,Databricks-Runtime/16.1.x-scala2.12


In [0]:
%sql
SELECT * 
FROM delta.`/mnt/silver-ankit/sales_order` 
VERSION AS OF 2;


OrderID,CustomerID,ProductID,OrderDate,Quantity,UnitPrice,TotalAmount,OrderStatus,LastUpdated,_rescued_data,IngestedAt
1,101,201,2025-01-01T10:30:00Z,2,50.00,100.00,Pending,2025-01-01T10:30:00Z,null,2025-01-15T06:40:11.095Z
2,102,202,2025-01-02T14:15:00Z,1,75.50,75.50,Completed,2025-01-02T14:15:00Z,null,2025-01-15T06:40:11.095Z
3,103,203,2025-01-03T09:45:00Z,5,20.00,100.00,Shipped,2025-01-03T09:45:00Z,null,2025-01-15T06:40:11.095Z
4,104,204,2025-01-04T16:00:00Z,3,150.75,452.25,Cancelled,2025-01-04T16:00:00Z,null,2025-01-15T06:40:11.095Z
5,105,205,2025-01-05T11:20:00Z,10,10.50,105.00,Pending,2025-01-05T11:20:00Z,null,2025-01-15T06:40:11.095Z
6,106,206,2025-01-06T18:30:00Z,4,99.99,399.96,Completed,2025-01-06T18:30:00Z,null,2025-01-15T06:40:11.095Z
7,107,207,2025-01-07T13:10:00Z,7,40.00,280.00,Shipped,2025-01-07T13:10:00Z,null,2025-01-15T06:40:11.095Z
8,108,208,2025-01-08T15:45:00Z,6,25.25,151.50,Pending,2025-01-08T15:45:00Z,null,2025-01-15T06:40:11.095Z
9,109,209,2025-01-09T10:00:00Z,12,8.99,107.88,Completed,2025-01-09T10:00:00Z,null,2025-01-15T06:40:11.095Z
10,10,110,2025-01-10T12:00:00Z,3,60.00,180.00,Pending,2025-01-10T12:00:00Z,null,2025-01-15T06:40:11.095Z
